In [1]:
import pandas as pd
import numpy as np

import sys
sys.path.insert(0, "../scripts")
import helpers as h
import window_detect2 as w
import scores as s
import ml_window as m
from icecream import ic

import pickle

import plotly.graph_objects as go

In [2]:
a00, a01 = h.import_desired_data("A", "15T")
window_sched = np.array(a01["Window Open"])

In [3]:
with open('../constants/data_combos_230638.pkl', 'rb') as handle:
    test_dict = pickle.load(handle)

In [4]:
for k, v in test_dict.items():
    # print(k)
    obj = m.ML_Window_Detect(v["data"], window_sched)
    obj.run_all()
    test_dict[k]["accuracy"] = obj.accuracy
    test_dict[k]["choices"] = obj.choices

In [5]:
def get_guess_times(test_dict_item):
    choice_list = pd.Series(test_dict_item["choices"])
    mask = choice_list.shift() != choice_list
    guesses = choice_list[mask].index
    guess_times = a01["DateTime"][guesses]

    return guess_times

for k in test_dict.keys():
    g = get_guess_times(test_dict[k])
    s1 = s.Scores(exp=a01, choices=test_dict[k]["choices"], guess_times=g,)
    
    test_dict[k]["custom_report"] = s1.calc_hit_scores()

# with open("../constants/combo_results_230638.pkl", "wb") as fp:
#     pickle.dump(test_dict, fp )

In [6]:
for k, v in test_dict.items():
    print(k, v["custom_report"].loc["(hits + near hits)/actions"][0])

dt_meas 3.231
dt_sim 1.462
dt_amb 0.231
dt_meas+dt_sim+dt_amb 3.308
meas 1.846
amb 1.538
sim 1.538
meas+amb+sim 1.231
dt_meas+dt_sim 2.538
dt_meas+dt_amb 3.308
dt_meas+meas 1.769
dt_meas+amb 2.846
dt_meas+sim 2.385
dt_sim+dt_amb 4.846
dt_sim+meas 2.231
dt_sim+amb 2.0
dt_sim+sim 2.077
dt_amb+meas 4.615
dt_amb+amb 4.923
dt_amb+sim 5.385
meas+amb 1.692
meas+sim 1.923
amb+sim 1.308
amb_minus_meas 3.462
meas_minus_amb 3.462
abs_amb_meas_diff 3.692


In [7]:
df = pd.DataFrame(s1.scores.to_list(), columns=["Exact", "Nearest", "Distance"])
df

AttributeError: 'dict' object has no attribute 'to_list'

In [ ]:
s1.guess_times.index

Int64Index([  0,   4,   8,   9,  11,  13,  14,  15,  19,  20,  21,  31,  32,
             35,  45,  52,  53,  55,  56,  75,  76,  89,  93, 103, 105, 106,
            107, 109, 111, 119, 123, 125, 127, 135, 137, 139, 140, 145, 147,
            148, 151, 153, 156, 159, 161, 171, 173, 179, 199, 200, 205, 211,
            239, 242, 244, 247, 248, 250, 251, 258, 291, 292, 306, 315, 319,
            323, 331, 334, 335, 339, 340, 344, 345, 347, 348, 351, 352, 360,
            371, 379, 383],
           dtype='int64')

In [ ]:
shift = s1.exp["Window Open"].shift() != s1.exp["Window Open"]
change_indices = np.where(shift==True)[0]
change_indices

array([  0,   2,  21,  61, 120, 127, 137, 158, 194, 208, 227, 255, 338])

In [ ]:
pred_change = {change_ix: time_ix for  change_ix, time_ix in  enumerate(s1.guess_times.index)}
pred_change

true_change = {change_ix: time_ix for  change_ix, time_ix in  enumerate(change_indices)
}
true_change

{0: 0,
 1: 2,
 2: 21,
 3: 61,
 4: 120,
 5: 127,
 6: 137,
 7: 158,
 8: 194,
 9: 208,
 10: 227,
 11: 255,
 12: 338}

In [ ]:
res = {}
for k, v in pred_change.items():
    nearest_ix = h.find_nearest(list(true_change.values()), v)
    distance = v - nearest_ix
    res[k] = {}
    res[k]["match?"] = True if distance == 0 else False
    res[k]["nearest_ix"] = nearest_ix
    res[k]["distance"] = distance





In [ ]:

hits = 0
near_hits = 0
miss = 0
for k, v in res.items():
    if v["match?"]:
        hits+=1
    elif v["distance"] <=2:
        near_hits+=1
    else:
        miss+=1





In [ ]:
scores = {
    "hits": 0, "near_hits": 0, "miss": 0
}

for k, v in res.items():
    if v["match?"]:
        scores["hits"]+=1
    elif v["distance"] <=2:
        scores["near_hits"]+=1
    else:
        scores["miss"]+=1


In [ ]:
len(scores)

3

In [ ]:
ic(hits, near_hits, miss) # if hits + near hits > actions, then model is flawed..

ic| hits: 4, near_hits: 44, miss: 33


(4, 44, 33)

In [ ]:
res

{0: {'match?': True, 'nearest_ix': 0, 'distance': 0},
 1: {'match?': False, 'nearest_ix': 2, 'distance': 2},
 2: {'match?': False, 'nearest_ix': 2, 'distance': 6},
 3: {'match?': False, 'nearest_ix': 2, 'distance': 7},
 4: {'match?': False, 'nearest_ix': 2, 'distance': 9},
 5: {'match?': False, 'nearest_ix': 21, 'distance': -8},
 6: {'match?': False, 'nearest_ix': 21, 'distance': -7},
 7: {'match?': False, 'nearest_ix': 21, 'distance': -6},
 8: {'match?': False, 'nearest_ix': 21, 'distance': -2},
 9: {'match?': False, 'nearest_ix': 21, 'distance': -1},
 10: {'match?': True, 'nearest_ix': 21, 'distance': 0},
 11: {'match?': False, 'nearest_ix': 21, 'distance': 10},
 12: {'match?': False, 'nearest_ix': 21, 'distance': 11},
 13: {'match?': False, 'nearest_ix': 21, 'distance': 14},
 14: {'match?': False, 'nearest_ix': 61, 'distance': -16},
 15: {'match?': False, 'nearest_ix': 61, 'distance': -9},
 16: {'match?': False, 'nearest_ix': 61, 'distance': -8},
 17: {'match?': False, 'nearest_ix':

In [ ]:
test_dict["abs_amb_meas_diff"]["custom_report"]

,results
hits/guesses,0.037
hits/actions,0.231
(hits + near hits)/guesses,0.160
(hits + near hits)/actions,1.000
misses/guesses,0.840
number of actions,13.000
number of guesses,81.000
hit,3.000
near_hit,10.000
miss,68.000


In [ ]:
for k, v in test_dict.items():
    print(k, v["custom_report"].loc["(hits + near hits)/actions"][0])

dt_meas 153.8
dt_sim 69.19999999999999
dt_amb 7.7
dt_meas+dt_sim+dt_amb 92.30000000000001
meas 69.19999999999999
amb 53.800000000000004
sim 53.800000000000004
meas+amb+sim 38.5
dt_meas+dt_sim 107.69999999999999
dt_meas+dt_amb 130.8
dt_meas+meas 53.800000000000004
dt_meas+amb 123.10000000000001
dt_meas+sim 107.69999999999999
dt_sim+dt_amb 100.0
dt_sim+meas 92.30000000000001
dt_sim+amb 76.9
dt_sim+sim 92.30000000000001
dt_amb+meas 107.69999999999999
dt_amb+amb 153.8
dt_amb+sim 146.2
meas+amb 53.800000000000004
meas+sim 61.5
amb+sim 61.5
amb_minus_meas 92.30000000000001
meas_minus_amb 92.30000000000001
abs_amb_meas_diff 100.0
